In [1]:
import datajoint as dj
import os
from pathlib import Path

# set dirs
base_dir = Path('/Users/rly/Documents/NWB/spyglass-workspace') # change this to your desired directory
# base_dir = Path('/hdd/dj') # change this to your desired directory
if (base_dir).exists() is False:
    os.mkdir(base_dir)
raw_dir = base_dir / 'raw'
if (raw_dir).exists() is False:
    os.mkdir(raw_dir)
analysis_dir = base_dir / 'analysis'
if (analysis_dir).exists() is False:
    os.mkdir(analysis_dir)
recording_dir = base_dir / 'recording'
if (recording_dir).exists() is False:
    os.mkdir(recording_dir)
sorting_dir = base_dir / 'sorting'
if (sorting_dir).exists() is False:
    os.mkdir(sorting_dir)
waveforms_dir = base_dir / 'waveforms'
if (waveforms_dir).exists() is False:
    os.mkdir(waveforms_dir)
tmp_dir = base_dir / 'tmp'
if (tmp_dir).exists() is False:
    os.mkdir(tmp_dir)
kachery_cloud_dir = base_dir / '.kachery_cloud'
if (kachery_cloud_dir).exists() is False:
    os.mkdir(kachery_cloud_dir)

# set dj config
dj.config['database.host'] = 'localhost'
dj.config['database.user'] = 'root'
dj.config['database.password'] = 'tutorial'
dj.config['database.port'] = 3306
dj.config['stores'] = {
  'raw': {
    'protocol': 'file',
    'location': str(raw_dir),
    'stage': str(raw_dir)
  },
  'analysis': {
    'protocol': 'file',
    'location': str(analysis_dir),
    'stage': str(analysis_dir)
  }
}

# set env vars
os.environ['SPYGLASS_BASE_DIR'] = str(base_dir)
os.environ['SPYGLASS_RECORDING_DIR'] = str(recording_dir)
os.environ['SPYGLASS_SORTING_DIR'] = str(sorting_dir)
os.environ['SPYGLASS_WAVEFORMS_DIR'] = str(waveforms_dir)
os.environ['SPYGLASS_TEMP_DIR'] = str(tmp_dir)
os.environ['KACHERY_CLOUD_DIR'] = str(kachery_cloud_dir)

os.environ['DJ_SUPPORT_FILEPATH_MANAGEMENT'] = 'TRUE'

dj.config["enable_python_native_blobs"] = True

import spyglass as sg
import spyglass.common as sgc
import spyglass.data_import as sgdi
import spyglass.spikesorting as sgs
import spyglass.decoding as sgd

[2022-10-20 12:06:02,206][INFO]: Connecting root@localhost:3306
[2022-10-20 12:06:02,288][INFO]: Connected root@localhost:3306
/home/kyu/miniconda3/envs/spyglass/lib/python3.8/site-packages/position_tools/core.py:3: DeprecationWarning: Please use `gaussian_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter1d


In [2]:
sgc.DataAcquisitionDeviceSystem().delete(safemode=False)
sgc.DataAcquisitionDeviceAmplifier().delete(safemode=False)
sgc.DataAcquisitionDevice().delete(safemode=False)
sgc.Institution().delete(safemode=False, force_parts=True)
sgc.Lab().delete(safemode=False)
sgc.LabTeam().delete(safemode=False)
sgc.LabMember().delete(safemode=False)
# sgc.Subject().delete(safemode=False)
sgc.Session().delete(safemode=False)
sgc.Nwbfile().delete(safemode=False)
sgc.Nwbfile().cleanup(delete_files=True)
sgc.Probe().delete(safemode=False)
sgs.SortGroup().delete(safemode=False)
sgs.SortInterval().delete(safemode=False)
sgs.SpikeSorterParameter().delete(safemode=False)
sgs.SpikeSortingPreprocessingParameter().delete(safemode=False)
sgs.SpikeSortingRecordingSelection().delete(safemode=False)
sgs.ImportedSpikeSorting().delete(safemode=False, force_parts=True)
sgs.Curation().delete(safemode=False)


[2022-10-20 12:06:22,543][INFO]: Deleting 0 rows from `common_device`.`data_acquisition_device`
[2022-10-20 12:06:22,558][INFO]: Deleting 0 rows from `common_lab`.`institution`
[2022-10-20 12:06:22,571][INFO]: Deleting 0 rows from `common_lab`.`lab`
[2022-10-20 12:06:22,591][INFO]: Deleting 0 rows from `common_lab`.`lab_team`
[2022-10-20 12:06:22,605][INFO]: Deleting 0 rows from `common_lab`.`lab_member`
[2022-10-20 12:06:22,618][INFO]: Deleting 0 rows from `common_subject`.`subject`
[2022-10-20 12:06:22,632][INFO]: Deleting 0 rows from `common_session`.`_session`
[2022-10-20 12:06:22,648][INFO]: Deleting 0 rows from `common_nwbfile`.`nwbfile`
0it [00:00, ?it/s]
[2022-10-20 12:06:22,710][INFO]: Deleting 0 rows from `common_device`.`probe`
[2022-10-20 12:06:22,723][INFO]: Deleting 0 rows from `spikesorting_recording`.`sort_group`
[2022-10-20 12:06:22,737][INFO]: Deleting 0 rows from `spikesorting_recording`.`sort_interval`
[2022-10-20 12:06:22,755][INFO]: Deleting 0 rows from `spikesort

1

In [3]:
sgc.Subject()

subject_id,age,description,genotype,sex,species
My Subject,6 months,None,WT (RRID:IMSR_JAX:000664),F,Mus musculus


In [4]:
# Define the name of the file that you copied and renamed; make sure it's something unique. 
# nwb_file_name = 'sub-CuffLinks_ses-CuffLinks-LT03-g3_behavior+ecephys.nwb'
nwb_file_name = 'sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys.nwb'

filename, file_extension = os.path.splitext(nwb_file_name)
# This is a copy of the original nwb file, except it doesn't contain the raw data (for storage reasons)
nwb_file_name2 = filename + '_' + file_extension

In [5]:
sgdi.insert_sessions(nwb_file_name)

/Users/rly/mambaforge/envs/spyglass/lib/python3.9/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'core' version 2.5.0 because version 2.4.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/Users/rly/mambaforge/envs/spyglass/lib/python3.9/site-packages/hdmf/utils.py:467: PendingDeprecationWarning: call_docval_func will be deprecated in a future version of HDMF. Instead of using call_docval_func, call the function directly with the kwargs. Please note that call_docval_func removes all arguments not accepted by the function's docval, so if you are passing kwargs that includes extra arguments and the function's docval does not allow extra arguments (allow_extra=True is set), then you will need to pop the extra arguments out of kwargs before calling the function.
  warnings.warn("call_docval_func will be deprecated in a future version of HDMF. Instead of using call_docval_func, "
/Users/rly/mambafo

Creating a copy of NWB file sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys.nwb with link to raw ephys data: sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb


/Users/rly/mambaforge/envs/spyglass/lib/python3.9/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'core' version 2.5.0 because version 2.4.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/Users/rly/mambaforge/envs/spyglass/lib/python3.9/site-packages/hdmf/utils.py:467: PendingDeprecationWarning: call_docval_func will be deprecated in a future version of HDMF. Instead of using call_docval_func, call the function directly with the kwargs. Please note that call_docval_func removes all arguments not accepted by the function's docval, so if you are passing kwargs that includes extra arguments and the function's docval does not allow extra arguments (allow_extra=True is set), then you will need to pop the extra arguments out of kwargs before calling the function.
  warnings.warn("call_docval_func will be deprecated in a future version of HDMF. Instead of using call_docval_func, "
/Users/rly/mambafo

Populate Session...
/Users/rly/Documents/NWB/spyglass-workspace/raw/sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb
Institution...

Lab...

LabMember...
Please add the Google user ID for Grad Student One in the LabMember.LabMemberInfo table if you want to give them permission to manually curate sortings by this team.


/Users/rly/Documents/NWB/spyglass/src/spyglass/common/common_lab.py:120: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if not query:



Subject...
Subject exists in database with subject ID 'My Subject' but its properties:
{'subject_id': 'My Subject', 'age': '6 months', 'description': None, 'genotype': 'WT (RRID:IMSR_JAX:000664)', 'sex': 'F', 'species': 'Mus musculus'}
differ from the properties read from the file:
{'subject_id': 'My Subject', 'age': '7 months', 'description': None, 'genotype': 'WT (RRID:IMSR_JAX:000664)', 'sex': 'F', 'species': 'Mus musculus'}

DataAcquisitionDevice...
No conforming data acquisition device metadata found.

CameraDevice...
No conforming camera device metadata found.

Probe...
No conforming probe metadata found.

Skipping Apparatus for now...
IntervalList...
No epochs found in NWB file.

Populate ExperimenterList...

Populate Raw...
Importing raw data: Sampling rate:	30000.0 Hz
Number of valid intervals:	1

Populate Electrode...

Populate SampleCount...
Unable to import SampleCount: no data interface named "sample_count" found in sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys

In [6]:
sgs.ImportedSpikeSorting()

nwb_file_name name of the NWB file,units_object_id
sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb,d4ed00bc-40af-46d6-add2-d7b94a34477d


In [7]:
sgc.Subject()

*curation_id   parent_curatio automatic_ curation_l merge_grou quality_me description    time_of_creati
+------------+ +------------+ +--------+ +--------+ +--------+ +--------+ +------------+ +------------+
041b59d3-4170- None           =BLOB=     =BLOB=     =BLOB=     =BLOB=     from ImportedS 1666292823    
 (Total: 1)

In [ ]:
sgc.Session()

In [ ]:
sgs.ImportedSpikeSorting()

In [ ]:
sgs.Curation()

In [8]:
sgs.Curation.ImportedSpikeSorting()

curation_id,nwb_file_name name of the NWB file
041b59d3-4170-42c5-ac62-efa4e37340ed,sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb


In [9]:
sgc.RawPosition()

nwb_file_name name of the NWB file,interval_list_name descriptive name of this interval list,raw_position_object_id the object id of the spatial series for this epoch in the NWB file
sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb,pos 0 valid times,dfb5f47d-66d8-49e8-82e5-ebb6086eb59c


In [11]:
sgc.Session()

nwb_file_name name of the NWB file,subject_id,institution_name,lab_name,session_id,session_description,session_start_time,timestamps_reference_time,experiment_description
sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys_.nwb,AppleBottom,Stanford University,Lisa Giocomo,AppleBottomDY20-g3,Open Field,2022-03-09 15:20:14,2022-03-09 15:20:14,"Entorhinal cortex spike times and LFP during linear track, alternation task, open field, and pre + post track rest in WT mice"


In [9]:
import pynwb
with pynwb.NWBHDF5IO(raw_dir / 'sub-AppleBottom_ses-AppleBottom-DY20-g3_behavior+ecephys.nwb', mode='r', load_namespaces=True) as io:
    nwbf = io.read()
    print(nwbf.units)
    